# EE194 Lab 0: Chisel - Part 1: Introduction to Scala, Chisel & Chisel Combinational Logic

Adapted from [Scott Beamer](https://scottbeamer.net/)’s [CSE 228A - Agile Hardware Design](https://github.com/agile-hw) lecture demos + labs & [FreeChipsProject Chisel bootcamp](https://github.com/freechipsproject/chisel-bootcamp)

This series of labs aim to be a concentrated combination of both of the above resources. We expect this series of labs to take ~6-10 hours total, across all parts.

## A paradigm shift from EECS151...
Designing a large SoC in ~18 weeks means changing the way we think about large scale design. You might've done/heard the following about large scale designs in industry:

### The traditional "waterfall" method of design:
![](./images/trad-hw.svg)
* Complete each stage before moving on
* Late integration or changes to design during optimization requires changing everything -- design, spec, reimplmentation, reoptimize, reverify...

This is like a large construction project: You carefully plan to ensure everything goes smoothly, you construct lots of tests, double check everything before you start construction (because once you construct, you can't change it again/or it is very difficult to change it).

### The Agile method of design:
What if... 

a) It was easy to change a design? -- Change the cache capacity just by modifying 1 parameter.

b) Had a wealth of reuseable modules at your disposal that you could reuse across projects? -- Need a Page Table Walker? Bring it in from another project and instantiate it with 1 line.

c) Generate a memory controller dynamically at compile time based on parameters you've supplied... Without writing RTL that is specifically targeted for that 1 design specification.

d) Disconnect your RTL from the underlying implementation technology: 1 set of RTL, build for FPGA or ASIC without thinking about changing your SRAMs to FPGA friendly memories.

e) Rip out an old module, hack in a new one, be able to unit test immediately even if the rest of the hardware immediately using that module isn't built... 

Agile design is:
* Recognizing that the end goal (or the path to it) will always be changing & hard to set in stone/know in advance.
* Realizing that during the process of designing, the designer will need to constantly reassess and adapt/change their design.
* Given the above, recognizing that there needs to be a way to easily integrate & test new designs, be able to change large chunks of a design without having to rebuild the entire thing from a new spec.

![](./images/agile-hw.svg)

## Chisel: Constructing Hardware in a Scala Embedded Language
* A language built on top of Scala (essentially a series of Scala packages) that enables construction of synthesizable hardware designs.
* Aims to harness the benefits of a traditional object-oriented software programming language (Scala) and apply those benefits to designing (and dynamically generating) hardware.
* Chisel ≠ Scala!

**Scala:**
* Strongly statically typed language supporting both Object-Oriented and functional programming.
* Can be compiled to Java and run on the JVM, provides native interoperability with Java.

### Chisel's goal is:
* Allow you to improve your design through iteration/revision, without having to implement an overly ambitious initial design
* Allow you to better allocate design & effort once you can see how things start fitting together
* Design for reuse of components
* Design for readability 

### Useful references/resources:
* Scala syntax cheatsheet: https://docs.scala-lang.org/cheatsheets/index.html 
* Scala getting started guide: https://docs.scala-lang.org/tour/basics.html

* Chisel Cookbook: https://www.chisel-lang.org/docs/cookbooks
* Chisel Explanations: https://www.chisel-lang.org/docs/explanations
* Chisel API Documentation: https://www.chisel-lang.org/api/latest/chisel3/index.html
* Chisel Test GitHub/Docs: https://github.com/ucb-bar/chiseltest


# Notes/Context:
## Scala's execution mechanisms / Why we are on a Jupyter Notebook:
Scala typically has 2 execution mechanisms:
1. Compile -> Execute
* Compile Scala program and run on JVM -- This is what Chipyard does
* Code need to be structured in classes & there needs to be a `main` class
* Need a build tool such as `sbt` (Used by Chipyard) to compile
2. Read-Eval-Print Loop (REPL)
* Write & evaluate a single line at time
* What these Jupyter notebooks do
    * We wanted you all do be able to do the labs before getting BWRC accounts / not have to deal with immediately doing account setup

## Chisel Versioning
* You might see many versions of Chisel throughout Chipyard, looking at documentation, etc...
* This lab uses Chisel 3.6.1 - Chipyard is compatible with this version of Chisel, however, some projects have support for Chisel 6 and Chisel 7. For the most part, you don't need to worry about Chisel versioning. Existing code (including those in Chipyard) will automatically pull in the right versions.
* Be aware when reading documentation. Some of the docs supplied above are for Chisel 7, meaning some functions might not be available in Chisel 3. But as far as staff can tell, you shouldn't run into those issues for these labs.
* Relevant Chisel Versions:
    * 3.6.1 (June 2024) is the bridge from the old to the new
    * 6.7 (March 2025) is the most recent released version
    * 7 is under public development
    * There is no version 4, as it was skipped to make version jump apparent

## Debugging
* There is a (short) debugging guide in the `part0` folder. It is intended to be a "just-in-time" lab in the sense that you read it whenever you need it/tips on how debugging works in Chisel. It is not required and you may find yourself reaching the end of this series of exercises without needing to read it. 

If you do find yourself in that situation, go take a short read of it anyways, the "how to print stuff" information is useful for when you are working in Chipyard.

# Start!

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE" or `YOUR ACTION NEEDED HERE`.

If you see `???` right below `YOUR CODE HERE`, make sure to remove that after you have implemented your solution (and before you run the code block).

### Import the necessary Chisel dependencies. 
> There will be cells like these in every lab. Make sure you run them before proceeding to bring the Chisel Library into the Jupyter Notebook scope!

In [0]:
interp.configureCompiler(_.settings.processArguments(List("-Wconf:cat=deprecation:s"), true))
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/resource/chisel_deps.sc"))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

## Scala conditionals
> Practice with Scala's if/else. Return `"heads"` if `flip` is `true`, else return `"tails"`

In [ ]:
def scalaCondPractice(flip: Boolean): String = {
    // YOUR CODE HERE
    ???
}

In [0]:
assert(scalaCondPractice(true) == "heads")
assert(scalaCondPractice(false) == "tails")

## Scala Lang & Chisel Conventions
We won't be giving a comprehensive overview of the Scala language or Chisel here; For that you should look at the resources above as you go through the exercises.

Listed here are some conventions you should follow when using Chisel.

### Variables

#### `var` - **Mutable** variable (**discouraged**)
* Functions like variables in conventional languages

#### `val` - **Immutable** variable (**encouraged**)
* Almost always used for variables when using Chisel
* Allows compiler to safely perform more ambitious optimizations
* Can increase code clarity by renaming values each step of the way

In [ ]:
var mutableVariable = 0
mutableVariable = 194

val immutableVariable = 42

In [ ]:
// This will error!
immutableVariable = 290

## Chisel Types
* Chisel has its own variable types. **These are not the same as Scala types!** These represent hardware components as opposed to Scala types which represent a value in software.

### `Bool` - Single-bit logical signal (`.B`)
* ≠ Scala's Boolean
### `UInt` - Unsigned Integer (`.U`)
* You can set bit-width explicitly or let it be inferred
### `SInt` - Signed Integer (`.S`)
* 2's complement

In [ ]:
0.B // Chisel Bool
true // Scala Boolean
true.B // Chisel Bool

6 // Scala Int
6.U // Chisel unsigned 6
6.S // Chisel signed 6
6.U(8.W) // Chisel unsigned 6 represented in 8 bits (unused bits are 0)

val inferredUInt: UInt = 4.U
val inferredUIntEx2 = 5.U

val explicitUInt_8 = 10.U(8.W)

-2.S // Chisel -2 signed
-2.S(8.W) // Chise -2 signed number represented in 8 bits

### Other ways to express literals
We can prefix strings with encoding `h` (hex), `o` (octal), `b` (binary)
* Then, can add `.U`, `.S`, `.B` cast to `UInt`, `SInt`, `Bool`
* Can also break up long literals with `_` in the string

In [ ]:
"b1010".U
"ha".U
"h_dead_beef".U
"ha".U
"ha".U(8.W)
"ha".asUInt(8.W)

## Chisel Operators & Combinational Logic
Check the [Chisel Cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/blob/master/chisel_cheatsheet-0.5.3-beta.pdf) for a list of operators (first page, top of third column)
* Most of the operators you expect exists
* NOTE: some of them have different symbols than what you are probably used to: `===` for equality checking in Chisel... `==` is for Scala equality checking
    * If you use `==` to check equality between 2 Chisel Objects, it'll error
* Pay attention to the result widths
* Pay attention to the parameters of these operators -- On the cheat sheet `c, x, y` are Chisel types; `n, m` are Scala Ints
    * Similar to Verilog, you can't use an input wire as the argument in bit slicing (ie - the following Verilog is what you would be trying to write had you used a Chisel UInt where a Scala Int was supposed to go)
    ```verilog
    module illegal(
        input [31:0] data_in,
        input [7:0] start,
        input [7:0] finish,
        output [7:0] slice_out
    );
  
        assign slice_out = data_in[start : finish];
  
    endmodule
    ```
* Just like Verilog, manipulating Chisel types will need to be in a `Module`!

In [ ]:
val a = 7.U
val b = 5.U

a + b // this will error!

In [ ]:
class ChiselOperators extends Module {
    val io = IO(new Bundle {
        val out = Output(UInt(8.W))
        val out_buggy = Output(UInt(8.W))
        val a_b_same = Output(Bool())
    })

    val a = 1.U
    val b = 2.U

    // YOUR CODE HERE
    io.out := a + b // This is actually a buggy implementation -- why? Find the bug and fix it!

    val c = 7.U
    val d = 5.U

    // YOUR CODE HERE
    // HINT! What is c + d supposed to be? Look at the "Width" column on the Chisel cheatsheet for the "+" operator.
    io.out_buggy := c + d
    // HINT! What operator should you actually be using instead of "+"?

    // YOUR ACTION NEEDED HERE
    io.a_b_same := (a == b) // This will error since we are trying to compare 2 Chisel types -- Try to comment this out and run the code again!

    io.a_b_same := (a === b)
    
    printf(p"out: ${io.out}, out_buggy: ${io.out_buggy}, a_b_same: ${io.a_b_same}\n") 
}

In [ ]:
def testChiselOperator: Boolean = {
    test(new ChiselOperators) { c =>
        c.io.out.expect(3.U)
        c.clock.step(1)
    }
    true
}
assert(testChiselOperator)

### More Chisel Operators: Combinational Logic
> Assign the boolean expression: `(a OR b) AND (NOT c)` to the module's output. 

In [ ]:
class CombLogic extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Input(Bool())
        val out = Output(Bool())
    })
    
    // YOUR CODE HERE
    ???
    
    // We can print state like this everytime `step()` is called in our test
    printf(p"a: ${io.a}, b: ${io.b}, c: ${io.c}, out: ${io.out}\n")
}

## Chisel Test

Documentation is here if you need it: https://github.com/ucb-bar/chiseltest; feel free to parrot the tests in the previous examples. If you need more help understanding how Chisel Tests work: https://github.com/agile-hw/lectures/blob/main/02-hello/lec02-hello.ipynb (Search: "Brief ChiselTest Intro")

TLDR:
* `poke` - Set value of wire
* `peek` - Read value of wire
* `expect` - read value & compare (assert)
* `clock.step(x)` - ticks the clock x times

### What is Chisel Test?
* By default, Chisel Test uses the [Treadle](https://github.com/chipsalliance/treadle) simulation engine, but it has native Verilator support as well.
* The closest thing to compare Chisel Tests to is a Verilog test bench you might've written in EECS151.
* You aren't running a binary -- You are essentially testing your circuit by driving individual wires... So Chisel Test is good for testing individual modules, but keep in mind this **will not** test/find issues when you start integrating, especially with existing Chipyard IP.
* Generally it isn't too productive to spend a whole ton of time writing Chisel Tests... It is more of a "sanity check" -- In line with the Agile Design principals, you should get the most basic implementation complete to a state where you can run a "hello world" RISC-V binary on it. Then define the workload you would eventually want your block to run in the RISC-V binary/Baremetal tests. It'll then fail horribly for the workload that you want it to eventually do -- but that's OK! You can then iterate towards that goal.

> Write your own test that tests `CombLogic` exhaustively for all input values `a, b, and c`. The module should return `true` if and only if all calls to `dut.io.out.expect(...)` succeed.


In [ ]:
def testCombLogic: Boolean = {
    test(new CombLogic) { dut =>
        
        // YOUR CODE HERE
        ???

    }
    true
}

In [ ]:
assert(testCombLogic)

### Where is my Verilog?!
Gimme Gimme Gimme (A Verilog Module)!

Hold your horses... We'll use this opportunity to discuss how your design goes from a `.scala` file to Verilog. 👻 Compilers! 👻

This is good information to know for when you have to go look at the Verilog that the Chisel you wrote generates and open a file up to a couple thousand lines of generated Verilog >_> Having this context will make it slightly more sane.

#### Chisel Tool Flow (Frontend)
* TLDR: The frontend flow for Chisel brings your design from a `.scala` file to a `.fir` file, which is an [intermediate representation](https://en.wikipedia.org/wiki/Intermediate_representation) of your circuit. (See graphic below)

The longer answer is that when you hit "Build" for your Chisel project, the build tool (SBT) brings in the Chisel Plugin/Library, then all of that is sent through the Scala Compiler (Scalac) to be converted to Java Byte Code. Then that byte code is run on the JVM, which elaborates your design using dependencies from the Chisel Plugin/Library. During elaboration, a Chisel IR is generated which takes the shape of a graph of the hardware you have described. This then is converted (or you can emit/dump Chisel IR, but that's not very helpful) to CHIRRTL (also known as "high-firrtl"). Then, CHIRRTL is descended down to the FIRRTL representation. This results in the `.fir` file.

![](./images/frontend.svg)

We can inspect the FIRRTL for our `CombLogic` Module:

In [ ]:
import chisel3.stage.ChiselStage
import scala.io.Source
import scala.util.{Try, Using}

object FirrtlMain extends App {
    (new ChiselStage).emitFirrtl(new CombLogic, args)
}

// Dump .fir file into output so it is easier to read here - If this errors the first time around, run this cell again
Using(Source.fromFile("./CombLogic.fir")) { source =>
    source.getLines.foreach(println)
}

#### Chisel Tool Flow (Backend)
To convert our FIRRTL representation into Verilog, we run the contents of the `.fir` file through the [FIRRTL compiler](https://github.com/chipsalliance/firrtl) (now known as CIRCT in newer versions of Chisel, but Chipyard hasn't upgraded yet :/)

![](./images/backend.svg)

This results in our Verilog:

In [ ]:
printVerilog(new CombLogic)

If you are curious about Chisel and how the elaboration & compilation works, check out these links:
* https://docs.google.com/presentation/d/1gMtABxBEDFbCFXN_-dPyvycNAyFROZKwk-HMcnxfTnU/edit?slide=id.p#slide=id.p
* https://www.youtube.com/watch?v=2-ZiXNd9wbc
* https://www.youtube.com/watch?v=qM9G0jr3rLY

^ From: https://stackoverflow.com/questions/58510182/developers-guide-for-chisel

The complete flow for `.scala` to independent VCS/Verilator full binary simulation is below:
![](./images/chisel_to_verilator_fullflow.png)

## Scala Conditionals in Chisel modules
> At hardware elaboration time, we can use Scala conditionals to change which hardware is created within a module. Implement the module such that if the `useAnd` argument is `true`, the generated hardware produces `a && b`, and otherwise produces `a || b`. The generated hardware should contain only `AND` logic or `OR` logic, but not both.

This is quite similar in functionality to `ifdef in Verilog, however, here you have the full power of the Scala programming language at your disposal to determine what blocks should be generated at elaboration time and what blocks shouldn't.

In [ ]:
class AndOrGenerationTime(useAnd: Boolean) extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val out = Output(Bool())
    })
    
    // YOUR CODE HERE
    ???
}

In [ ]:
def testAndOrGenerationTime(useAnd: Boolean): Boolean = {
    test(new AndOrGenerationTime(useAnd)) { dut =>
        for (a <- Seq(true, false)) {
            for (b <- Seq(true, false)) {
                dut.io.a.poke(a.B)
                dut.io.b.poke(b.B)
                if (useAnd) dut.io.out.expect((a && b).B)
                else        dut.io.out.expect((a || b).B)
            }
        }
    }
    true
}
assert(testAndOrGenerationTime(useAnd = true))
assert(testAndOrGenerationTime(useAnd = false))

You can see in the Verilog that only either the `AND` or `OR` logic is being generated:

In [ ]:
printVerilog(new AndOrGenerationTime(true))
println("\n")
printVerilog(new AndOrGenerationTime(false))

## Chisel Conditional in Chisel modules
> Generated hardware can use conditionals (i.e `Mux` or `when/elsewhen/otherwise`) to select signals. In this exercise, `useAnd` is an `Input` to the module. If `useAnd` is `true`, then the output `out` should be `a && b`, otherwise `a || b`. In this problem, both the logic for `a && b` and `a || b` hardware should be generated. Use a Chisel `Mux` for this exercise, as in the next one you will get to use a Chisel `when` statement.

FYI: If you want a Mux-tree / multi-input Mux, see: https://javadoc.io/doc/edu.berkeley.cs/chisel3_2.13/latest/chisel3/util/Mux1H$.html

In [ ]:
class AndOrRunTime extends Module {
    val io = IO(new Bundle {
        val a      = Input(Bool())
        val b      = Input(Bool())
        val useAnd = Input(Bool())
        val out    = Output(Bool())
    })
    
    // YOUR CODE HERE
    ???
}

In [ ]:
def testAndOrRunTime: Boolean = {
    test(new AndOrRunTime) { dut =>
        for (a <- Seq(true, false)) {
            for (b <- Seq(true, false)) {
                for (useAnd <- Seq(true, false)) {
                    dut.io.a.poke(a.B)
                    dut.io.b.poke(b.B)
                    dut.io.useAnd.poke(useAnd.B)
                    if (useAnd) dut.io.out.expect((a && b).B)
                    else        dut.io.out.expect((a || b).B)
                }
            }
        }
    }
    true
}
assert(testAndOrRunTime)

## Chisel's Last connect semantics
> When connecting Chisel components, the last connection made is the one that "wins" (exists in the generated hardware). In the module below, the default output is `5.U` because `out` is connected to `5.U` after `4.U`. Use a `when` statement to conditionally connect `8.U` to the output when the input `update` is set high, or keep the default connection when `update` is set low.

In [ ]:
class LastConnect extends Module {
    val io = IO(new Bundle {
        val update = Input(Bool())
        val out    = Output(UInt())
    })
    
    io.out := 4.U
    io.out := 5.U
    // YOUR CODE HERE
    ???
}

In [ ]:
def testLastConnect: Boolean = {
    test(new LastConnect) { dut =>
        dut.io.update.poke(true.B)
        dut.io.out.expect(8.U)
        
        dut.io.update.poke(false.B)
        dut.io.out.expect(5.U)
    }
    true
}
assert(testLastConnect)

## Simple ReLU
> Let's put together some of these techniques to build a more complicated module. A ReLU or rectified linear unit is a function used in ML. (https://en.wikipedia.org/wiki/Rectifier_(neural_networks))

> To combine everything we've learned so far we will slightly modify the function to saturate at a parameterized upper-bound of our choosing. The module will compute this function: `f(x, upper_bound) = max(0, min(x, upperBound))`.

> Here is an example where we parameterize `upperBound = 3`. Note the input `x` and output `y` will be of type SInt.

<img src="images/relu.png" style="width:60%;">

In [ ]:
class ReLU(upperBound: Int) extends Module {
    val io = IO(new Bundle {
        val x = Input(SInt(5.W))
        val y = Output(SInt(5.W))
    })
    // YOUR CODE HERE
    ???
}

### Testing ReLU
> A test for `ReLU` that tests `x` at input values `-1, 0, 1, 15`. The test (`testReLU`) is parameterized by `upperBound`, and you can assume `upperBound` is non-negative. The module should return `true` if and only if all calls to `dut.io.y.expect(...)` succeed.

In [ ]:
def testReLU(upperBound: Int): Boolean = {
    require(upperBound >= 0)
    test(new ReLU(upperBound)) { dut =>
        dut.io.x.poke(-1.S)
        dut.io.y.expect(0.S)

        dut.io.x.poke(0.S)
        dut.io.y.expect(0.S)

        if (upperBound < 1) {
            dut.io.x.poke(1.S)
            dut.io.y.expect(0.S)
        } else {
            dut.io.x.poke(1.S)
            dut.io.y.expect(1.S)
        }

        if (upperBound < 15) {
            dut.io.x.poke(15.S)
            dut.io.y.expect(upperBound.S)
        } else {
            dut.io.x.poke(15.S)
            dut.io.y.expect(15.S)
        }
    }
    true
}

In [ ]:
for(upperBound <- 0 until 16) {
    println(s"Testing ReLu, upperBound=$upperBound")
    assert(testReLU(upperBound))
}